# Compute the Storage Capacity of Matched Molecules

In [1]:
from mongoengine import connect
from ase.calculators.calculator import CalculationFailed
from cfree.store import MoleculeRecord
from cfree.descriptors import compute_wth2, count_h2_difference, saturate_molecule
from cfree.compute import compute_storage_energy
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
import numpy as np

Configuration

In [2]:
to_run_per_target = 4  # How many baseline structures to test
match_type = 'abs-cosine'

Connect to the database

In [3]:
client = connect(port=27894)
coll = client['cfree']['molecule_record']

## Load the Matches
They are stored in CSV files produced by Zhi Hong. Get the top molecules out of Each

In [4]:
relevant = pd.read_csv(f'matched/25apr23-partial-PRD/Known_Pareto-Relevant_ENA-molecules-{match_type}-all.csv')
top_relevant = relevant.sort_values('Cosine Similarity').groupby('Known Pareto Molecule').tail(to_run_per_target)
print(f'Loaded {len(top_relevant)} molecules')

Loaded 28 molecules


In [5]:
random = pd.read_csv(f'matched/25apr23-partial-PRD/Known_Pareto-Random_Valid-molecules-{match_type}-all.csv')
top_random = random.sort_values('Cosine Similarity').groupby('Known Pareto Molecule').tail(to_run_per_target)
print(f'Loaded {len(top_random)} molecules')

Loaded 28 molecules


## Get Some Baseline Structures
Randomly select records from the database

In [6]:
%%time
rng = np.random.RandomState(1)
random_smiles = sorted([x['identifier']['smiles'] for x in coll.find({'subsets': 'random-valid'}, projection=['identifier.smiles'])])
random_smiles = rng.choice(random_smiles, replace=False, size=(len(top_relevant),)).tolist()
print(f'Pulled {len(random_smiles)} random "valid" molecules from the database')

Pulled 28 random "valid" molecules from the database
CPU times: user 674 ms, sys: 454 ms, total: 1.13 s
Wall time: 6.61 s


In [7]:
%%time
rng = np.random.RandomState(1)
relevant_smiles = sorted([x['identifier']['smiles'] for x in coll.find({'subsets': 'relevant-ENA'}, projection=['identifier.smiles'])])
relevant_smiles = rng.choice(relevant_smiles, replace=False, size=(len(top_relevant),)).tolist()
print(f'Pulled {len(relevant_smiles)} random "relevant" molecules from the database')

Pulled 28 random "relevant" molecules from the database
CPU times: user 245 ms, sys: 39.4 ms, total: 284 ms
Wall time: 3.18 s


## Get their Smiles Strings
Look that up from the database

Get them via projection

In [8]:
def find_smiles(key):
    """Get the record matching a certain InChI Key and return the smiles"""
    return coll.find_one({'_id': key})['identifier']['smiles']

In [9]:
top_random.head(1)

,Known Pareto Molecule,Cosine Similarity,Random Valid Molecule
233,UFWIBTONFRDIAS-UHFFFAOYSA-N,0.526907,KYNSBQPICQTCGU-UHFFFAOYSA-N


In [10]:
for data in [top_relevant, top_random]:
    for col in data.columns[[0, 2]]:
        data[f'{col}-SMILES'] = data[col].apply(find_smiles)

In [11]:
data.head(1)

,Known Pareto Molecule,Cosine Similarity,Random Valid Molecule,Known Pareto Molecule-SMILES,Random Valid Molecule-SMILES
233,UFWIBTONFRDIAS-UHFFFAOYSA-N,0.526907,KYNSBQPICQTCGU-UHFFFAOYSA-N,c1ccc2ccccc2c1,C1=Cc2ccccc2OC1


## Compute the Storage Capacity
This is a simple calculation from the parsed string. Run it, then store the result in the database

Do it for the known molecules

In [12]:
def compute_wth2_if_needed(smiles: str) -> float:
    """Compute the wt%H2 of a molecule if we have not already
    
    Also store the result in the database if it's new.
    
    Args:
        smiles: SMILES string of the molecule in question
    Returns:
        The storage capacity (wt%H2)
    """
    # Get the document based on the inchi key
    key = Chem.MolToInchiKey(Chem.MolFromSmiles(smiles))
    record = coll.find_one({'_id': key}, projection=['property'])
    if record is None:
        return None
    
    # Check if property is set
    if 'wt%H2' not in record.get('property', {}):
        wt = compute_wth2(smiles)
        coll.update_one({'_id': key}, {'$set': {'property.wt%H2': wt}})
    else:
        wt = record['property']['wt%H2']
    return wt

In [13]:
compute_wth2_if_needed('c1nc[nH]n1')

5.517491899109785

In [14]:
known_molecules = pd.read_csv('../screen-search-space/to-compare.smi', names=['smiles'])

In [15]:
for smiles in known_molecules['smiles']:
    compute_wth2_if_needed(smiles)

Run it for the baselines

In [16]:
for smiles in tqdm(random_smiles + relevant_smiles):
    compute_wth2_if_needed(smiles)

100%|██████████| 56/56 [00:00<00:00, 83.91it/s] 


Run it for everyone

In [17]:
for smiles in tqdm(top_random.iloc[:, 4].values):
    compute_wth2_if_needed(smiles)

100%|██████████| 28/28 [00:00<00:00, 789.59it/s]


In [18]:
for smiles in tqdm(top_relevant.iloc[:, 4].values):
    compute_wth2_if_needed(smiles)

100%|██████████| 28/28 [00:00<00:00, 880.98it/s]


## Compute the Storage Energy
We'll use XTB to make it fast and generally accurate.

Make the functions first

In [19]:
def compute_eng_if_needed(smiles: str) -> float:
    """Compute the energy penalty of a molecule if we have not already
    
    Also store the result in the database if it's new.
    
    Args:
        smiles: SMILES string of the molecule in question
    Returns:
        Energy barrier in kJ/mol
    """
    # Get the document based on the inchi key
    key = Chem.MolToInchiKey(Chem.MolFromSmiles(smiles))
    record = coll.find_one({'_id': key}, projection=['property'])
    if record is None:
        return None
    
    # Check if property is set
    if 'storage_eng' not in record.get('property', {}):
        try:
            wt = compute_storage_energy(smiles)
        except CalculationFailed as exc:
            print(f'{smiles} failed: {exc}')
            return
        coll.update_one({'_id': key}, {'$set': {'property.storage_eng': wt}})
    else:
        wt = record['property']['storage_eng']
    return wt

Run it for the known molecules

In [20]:
for smiles in tqdm(known_molecules['smiles']):
    compute_eng_if_needed(smiles)

100%|██████████| 149/149 [00:00<00:00, 976.18it/s]


Run it for the baselines

In [21]:
for smiles in tqdm(relevant_smiles + random_smiles):
    compute_eng_if_needed(smiles)

 52%|█████▏    | 29/56 [01:00<00:56,  2.10s/it]

I.NC(=NCC1(c2ccc(F)cc2)CCCC1)NCCCN1CCOCC1 failed: xtb could not evaluate input


 82%|████████▏ | 46/56 [03:32<00:51,  5.20s/it]

Cc1ccnc(N2CCC(C(=O)N(CC3COC3)c3ccc(Cl)cc3)CC2)c1.O=C(C1CCN(c2cc3c(cn2)CCC3)CC1)N1CCOCc2cc(Cl)ccc21.O=C(C1CCN(c2cc3ccccc3cn2)CC1)N1CCOCc2cc(Cl)ccc21 failed: xtb could not evaluate input


[14:36:49] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [14:36:49] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit ERROR: [14:36:49] UFFTYPER: Unrecognized charge state for atom: 0
[14:36:49] UFFTYPER: Unrecognized charge state for atom: 0
RDKit WARNING: [14:36:49] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit ERROR: [14:36:49] UFFTYPER: Unrecognized charge state for atom: 0
[14:36:49] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[14:36:49] UFFTYPER: Unrecognized charge state for atom: 0
100%|██████████| 56/56 [05:58<00:00,  6.40s/it]

CCOC(=O)/C=C/C(=O)[O-].CCOC(=O)/C=C/C(=O)[O-].[Mg+2] failed: xtb could not evaluate input


Run it for everyone

In [22]:
for smiles in tqdm(top_random.iloc[:, 4].values):
    compute_eng_if_needed(smiles)

  7%|▋         | 2/28 [07:48<1:41:30, 234.26s/it]RDKit WARNING: [14:47:04] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:47:04] UFFTYPER: Warning: hybridization set to SP3 for atom 1
RDKit WARNING: [14:47:04] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[14:47:04] UFFTYPER: Warning: hybridization set to SP3 for atom 1
 14%|█▍        | 4/28 [09:33<50:58, 127.45s/it]  

CC(C)(C)NCc1cc(Nc2ccnc3cc(Cl)ccc23)c2c(c1O)CCCC2.O=P(O)(O)O.O=P(O)(O)O failed: xtb could not evaluate input


 21%|██▏       | 6/28 [41:00<3:03:20, 500.03s/it]RDKit WARNING: [15:20:16] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[15:20:16] UFFTYPER: Warning: hybridization set to SP3 for atom 1
RDKit WARNING: [15:20:16] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[15:20:16] UFFTYPER: Warning: hybridization set to SP3 for atom 1
 36%|███▌      | 10/28 [42:46<1:09:33, 231.88s/it]

CC(C)(C)NCc1cc(Nc2ccnc3cc(Cl)ccc23)c2c(c1O)CCCC2.O=P(O)(O)O.O=P(O)(O)O failed: xtb could not evaluate input


RDKit ERROR: [15:22:01] UFFTYPER: Unrecognized atom type: S_5+4 (1)
[15:22:01] UFFTYPER: Unrecognized atom type: S_5+4 (1)
 43%|████▎     | 12/28 [45:12<49:39, 186.24s/it]  RDKit WARNING: [15:24:28] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[15:24:28] UFFTYPER: Warning: hybridization set to SP3 for atom 1
RDKit WARNING: [15:24:28] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[15:24:28] UFFTYPER: Warning: hybridization set to SP3 for atom 1
 46%|████▋     | 13/28 [46:57<43:04, 172.28s/it]

CC(C)(C)NCc1cc(Nc2ccnc3cc(Cl)ccc23)c2c(c1O)CCCC2.O=P(O)(O)O.O=P(O)(O)O failed: xtb could not evaluate input


RDKit WARNING: [15:26:12] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[15:26:12] UFFTYPER: Warning: hybridization set to SP3 for atom 4
RDKit WARNING: [15:26:12] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[15:26:12] UFFTYPER: Warning: hybridization set to SP3 for atom 4
100%|██████████| 28/28 [58:55<00:00, 126.27s/it]


In [23]:
for smiles in tqdm(top_relevant.iloc[:, 4].values):
    compute_eng_if_needed(smiles)

100%|██████████| 28/28 [30:53<00:00, 66.18s/it]  
